# Data Load

In [1]:
import config
import swifter
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.origin_data)

extension : .pickle
Loaded 10317 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_doc2vec.pickle


In [3]:
df.head()

,content,pos
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 먹음]"


### pos concatenate

In [4]:
df['pos_str'] = df.pos.swifter.apply(lambda x: " ".join(x))

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

In [5]:
df.head()

,content,pos,pos_str
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반]",나 중간고사 반
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]",중간고사 점수 내 반 제일 받음
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]",나 반 중간 성적 제일 좋아
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]",우리 반 내 시험 제일 봤다
4,중간고사 반 1등 먹음,"[중간고사, 반, 먹음]",중간고사 반 먹음


# 

# Tokenizer

### user defined symbols

In [6]:
from pprint import pprint

In [7]:
user_defined_symbols = [
    "<pad>",
    "<unk>",
    "<cls>",
    "<sep>",
    "<mask>",
    "<bos>",
    "<eos>",
    "<tsep>",
    "<name>",
    "<url>",
    "<file>",
    "<image>",
    "<video>",
    "<location>",
]

In [8]:
user_defined_symbols += [f"<user{i}>" for i in range(10)]
user_defined_symbols += [f"<unk{i}>" for i in range(10)]
unused_token_num = 100
unused_list = [f"<unused{i}>" for i in range(unused_token_num)]
whole_user_defined_symbols = user_defined_symbols + unused_list

In [9]:
pprint(user_defined_symbols)

['<pad>',
 '<unk>',
 '<cls>',
 '<sep>',
 '<mask>',
 '<bos>',
 '<eos>',
 '<tsep>',
 '<name>',
 '<url>',
 '<file>',
 '<image>',
 '<video>',
 '<location>',
 '<user0>',
 '<user1>',
 '<user2>',
 '<user3>',
 '<user4>',
 '<user5>',
 '<user6>',
 '<user7>',
 '<user8>',
 '<user9>',
 '<unk0>',
 '<unk1>',
 '<unk2>',
 '<unk3>',
 '<unk4>',
 '<unk5>',
 '<unk6>',
 '<unk7>',
 '<unk8>',
 '<unk9>']


# 

### train

In [10]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

In [11]:
tokenizer = Tokenizer(BPE(unk_token="<unk>"))

In [12]:
print(f"tokenizer : {tokenizer}")

tokenizer : <tokenizers.Tokenizer object at 0x107008800>


In [13]:
from tokenizers import normalizers
from tokenizers.normalizers import NFKC, BertNormalizer

In [14]:
n1 = NFKC()
n2 = BertNormalizer(
    clean_text=False,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False,
)

In [15]:
tokenizer.normalizer = normalizers.Sequence([n1, n2])

In [16]:
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Metaspace

In [17]:
tokenizer.pre_tokenizer = pre_tokenizers.Sequence(
    [Metaspace(replacement="_", add_prefix_space=True)]
)

In [18]:
from tokenizers.trainers import BpeTrainer

In [19]:
def gen():
    for row in df.pos_str:
        yield row

In [20]:
trainer = BpeTrainer(
    vocab_size=config.vocab_size,
    special_tokens=whole_user_defined_symbols,
)
tokenizer.train_from_iterator(gen(), trainer)

### test

In [21]:
output = tokenizer.encode("본 고안은 이러한 특성을 이용해 사용한다.")

In [22]:
print(output.ids)

[1737, 5545, 951, 1346, 494, 1276, 3100, 748, 952, 1997, 1282, 2161, 3529, 1]


In [23]:
tokenizer.decode(output.ids)

'_본 _고안 은 _이 러 한 _특 성 을 _이용 해 _사용 한다'

In [24]:
from tokenizers import decoders

In [25]:
tokenizer.decoder = decoders.BPEDecoder(suffix="_")
tokenizer.decode(output.ids).strip()

'본 고안은 이러한 특성을 이용해 사용한다'

# 

### convert transformers tokenize

In [38]:
from transformers import GPT2TokenizerFast, AutoTokenizer

In [27]:
fast_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

In [28]:
fast_tokenizer.pad_token = "<pad>"
fast_tokenizer.unk_token = "<unk>"
fast_tokenizer.cls_token = "<cls>"
fast_tokenizer.sep_token = "<sep>"
fast_tokenizer.mask_token = "<mask>"
fast_tokenizer.bos_token = "<bos>"
fast_tokenizer.eos_token = "<eos>"

In [29]:
special_tokens_dict = {"additional_special_tokens": user_defined_symbols}
fast_tokenizer.add_special_tokens(special_tokens_dict)

0

In [30]:
e = fast_tokenizer.encode("<user1>'테스트용'으로 \"잘\" 되는지 보고 있다<sep>")

In [31]:
print(e)
print(fast_tokenizer.decode(e))

[15, 134, 1, 1199, 1823, 924, 1, 3122, 134, 1, 973, 1, 4432, 1026, 1536, 2520, 3]
<user1> <unk>테스트용<unk>으로 <unk>잘<unk> 되는지 보고 있다<sep>


### remove cls sep

In [32]:
from tokenizers.processors import TemplateProcessing

In [33]:
fast_tokenizer.post_processor = TemplateProcessing(
    single="<cls> $A <sep>",
    pair="<cls> $A <sep> $B:1 <sep>:1",
    special_tokens=[(t, i) for i, t in enumerate(user_defined_symbols)],
)

In [34]:
e = fast_tokenizer.encode("<user1>'테스트용'으로 \"잘\" 되는지 보고 있다<sep>")

In [35]:
print(e)
print(fast_tokenizer.decode(e))

[15, 134, 1, 1199, 1823, 924, 1, 3122, 134, 1, 973, 1, 4432, 1026, 1536, 2520, 3]
<user1> <unk>테스트용<unk>으로 <unk>잘<unk> 되는지 보고 있다<sep>


### save

In [36]:
fast_tokenizer.save_pretrained(config.save_path)

('/Volumes/GoogleDrive/내 드라이브/MemeProject/model/tokenizer/tokenizer_config.json',
 '/Volumes/GoogleDrive/내 드라이브/MemeProject/model/tokenizer/special_tokens_map.json',
 '/Volumes/GoogleDrive/내 드라이브/MemeProject/model/tokenizer/vocab.json',
 '/Volumes/GoogleDrive/내 드라이브/MemeProject/model/tokenizer/merges.txt',
 '/Volumes/GoogleDrive/내 드라이브/MemeProject/model/tokenizer/added_tokens.json',
 '/Volumes/GoogleDrive/내 드라이브/MemeProject/model/tokenizer/tokenizer.json')

# 

### Test

In [39]:
t = AutoTokenizer.from_pretrained(config.save_path)

In [40]:
e = t("본 고안은 이러한 특성을 이용해 사용한다.")
print(e)
print(t.decode(e['input_ids']))

{'input_ids': [1737, 5545, 951, 1346, 494, 1276, 3100, 748, 952, 1997, 1282, 2161, 3529, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
 본 고안은 이러한 특성을 이용해 사용한다<unk>
